# Bước 1: import thư viện


In [ ]:
!pip install scikeras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [ ]:
import pandas as pd #đọc dữ liệu
import numpy as np #xử lý dữ liệu
import matplotlib.pyplot as plt #vẽ biểu đồ
from sklearn.preprocessing import MinMaxScaler #chuẩn hóa dữ liệu
from keras.callbacks import ModelCheckpoint #lưu lại huấn luyện tốt nhất
from keras.callbacks import EarlyStopping #tránh học tủ
from tensorflow.keras.models import load_model #tải mô hình

#các lớp để xây dựng mô hình
from keras.models import Sequential #đầu vào
from keras.layers import LSTM #học phụ thuộc
from keras.layers import Dropout #tránh học tủ
from keras.layers import Dense #đầu ra

from sklearn.model_selection import GridSearchCV, TimeSeriesSplit #tìm kiếm siêu tham số
from scikeras.wrappers import KerasRegressor #tạo wrapper cho scikit-learn

#kiểm tra độ chính xác của mô hình
from sklearn.metrics import r2_score #đo mức độ phù hợp
from sklearn.metrics import mean_absolute_error #đo sai số tuyệt đối trung bình
from sklearn.metrics import mean_absolute_percentage_error #đo % sai số tuyệt đối trung bình

In [ ]:
#tạo quyền truy cập với google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# đọc dữ liệu từ file csv
df = pd.read_csv('/content/drive/MyDrive/dataset/Dữ liệu Lịch sử VNM 2014_2024.csv')

# lưu kết quả chạy được ra 1 list
mae_results = []

#định dạng cấu trúc thời gian
df["Ngày"] = pd.to_datetime(df.Ngày,format="%d/%m/%Y")

df1 = pd.DataFrame(df,columns=['Ngày','Đóng cửa'])
df1.index = df1.Ngày
df1.drop('Ngày',axis=1,inplace=True)


df1['Đóng cửa'] = df1['Đóng cửa'].str.replace(',', '').astype(float)


df1.shape

(2496, 1)

In [ ]:
data = df1.values

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)


In [ ]:
# Hàm tạo dataset với time_step
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

# Hàm tạo mô hình LSTM với tham số truyền vào
def create_model(units=128, dropout=0.5):
    model = Sequential()
    model.add(LSTM(units=units, input_shape=(None, 1), return_sequences=True))
    model.add(LSTM(units=units // 2))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_absolute_error', optimizer='adam')
    return model


# Thử nghiệm với các giá trị time_step khác nhau
time_steps = list(range(5,50,1))

results = []

for time_step in time_steps:
    X, Y = create_dataset(scaled_data, time_step)

    # Reshape X để phù hợp với LSTM [samples, time_steps, features]
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # Chia dữ liệu thành tập train và test (80/20)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    Y_train, Y_test = Y[:train_size], Y[train_size:]

    # Tạo model KerasRegressor
    model = KerasRegressor(model=create_model, verbose=0)

    # Tạo lưới các tham số cần tìm kiếm
    param_grid = {
        'model__units': [64,128,256],
        'model__dropout': [0.2,0.6],
        'batch_size':[16,32,64],
        'epochs':list(range(5,100,1))
    }

    # Tìm kiếm siêu tham số tốt nhất bằng GridSearchCV với TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=3)  # Chia dữ liệu theo thời gian
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=tscv, n_jobs=-1)
    grid_result = grid.fit(X_train, Y_train)

    # In ra kết quả
    print(f"Best result for time_step {time_step}: %f using %s" % (grid_result.best_score_, grid_result.best_params_))



    # Dự đoán và tính toán MAE trên tập test
    best_model = grid_result.best_estimator_
    Y_pred = best_model.predict(X_test)
    Y_test_scaled = scaler.inverse_transform([Y_test])
    Y_pred_scaled = scaler.inverse_transform(Y_pred.reshape(-1, 1))
    mae = mean_absolute_error(Y_test_scaled[0], Y_pred_scaled[:, 0])

    results.append((time_step, grid_result.best_params_['model__units'],grid_result.best_params_['model__dropout'],grid_result.best_params_['batch_size'],grid_result.best_params_['epochs'], mae))


# Lưu lại kết quả vào DataFrame
results_df = pd.DataFrame(results,columns=['time_steps', 'model__units','model__dropout','batch_size','epochs','MAE'])

# Lưu DataFrame vào file CSV
results_df.to_csv(f'/content/drive/MyDrive/Kết Quả/Ket_Qua_Tim_Kiem_Sieu_Tham_So_VNM_1_{time_step}_.csv', index=False)
# In kết quả tổng hợp
